# 환경 설정

In [1]:
# Google Drive 마운트
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install ultralytics -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 21.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 120.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 99.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 60.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 42.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 20.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.7/188.7 MB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 103.2 MB/s eta 0:00:00


In [ ]:
# !rm -rf /content/YOLO_safety
# print("🧹 YOLO_safety 폴더 완전 삭제 완료")

In [11]:
!rm -rf /content/YOLO_safety/train

# 데이터 압축 해제

In [3]:
# zip 파일 읽어서 압축 해제

import os
import zipfile

# 경로 설정
base_dir = "/content/YOLO_safety"   # colab 로컬 디렉토리
os.makedirs(base_dir, exist_ok=True)

zip_path = "/content/drive/MyDrive/AI/"

# 압축 해제 함수
def unzip_files(zip_path, target_dir):
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(target_dir)
    print(f"압축 해제 완료: {zip_path} -> {target_dir}")

# 압축 해제
file_name = "fin3.zip"
unzip_files(zip_path + file_name, base_dir)

# # 압축 해제
# file_name = "2_bbox_completed.zip"
# unzip_files(zip_path + file_name, base_dir)

압축 해제 완료: /content/drive/MyDrive/AI/fin3.zip -> /content/YOLO_safety


# JSON 라벨을 YOLO 포맷으로 변환

In [12]:
import json
from pathlib import Path

def _get_image_meta(data):
    """
    a 포맷: data['image']['filename'], data['image']['resolution'] = [W,H]
    b 포맷: data['images']['file_name'], data['images']['width'], data['images']['height']
    """
    filename = None
    W = H = None

    # a 스타일
    if isinstance(data.get("image"), dict):
        im = data["image"]
        if isinstance(im.get("filename"), str):
            filename = im["filename"]
        if isinstance(im.get("resolution"), list) and len(im["resolution"]) == 2:
            W, H = im["resolution"][0], im["resolution"][1]
        else:
            # 혹시 width/height 키가 있으면 그것도 수용
            if isinstance(im.get("width"), (int, float)) and isinstance(im.get("height"), (int, float)):
                W, H = im["width"], im["height"]

    # b 스타일
    if filename is None and isinstance(data.get("images"), dict):
        im = data["images"]
        if isinstance(im.get("file_name"), str):
            filename = im["file_name"]
        if isinstance(im.get("width"), (int, float)) and isinstance(im.get("height"), (int, float)):
            W, H = im["width"], im["height"]

    # 보정
    if isinstance(W, float): W = int(W)
    if isinstance(H, float): H = int(H)

    return filename, W, H

def _is_b_helmet_off(ann: dict) -> bool:
    """b 포맷의 헬멧 미착용: object_class==1 또는 categories[].value에 '안전모 미착용'."""
    oc = ann.get("object_class")
    if isinstance(oc, (int, str)) and str(oc).strip() == "1":
        return True
    cats = ann.get("categories")
    if isinstance(cats, list):
        for c in cats:
            if isinstance(c, dict):
                v = c.get("value")
                if isinstance(v, str):
                    vv = v.strip()
                    if vv == "안전모 미착용" or ("안전모" in vv and "미착용" in vv):
                        return True
    return False

def convert_box_json_to_yolo(json_dir, output_dir, class_map, log_path="convert_log.txt"):
    """
    box 또는 polygon 기반 JSON 라벨을 YOLO 포맷으로 변환하는 함수.
    - a 포맷: annotations[].class ∈ class_map
    - b 포맷: annotations[].object_class == 1 또는 categories[].value == '안전모 미착용' → class_map['08']로 매핑
    - polygon은 좌표의 min/max로 bounding box 추정
    - bbox 유효성 검사 포함
    - 객체가 없더라도 빈 .txt 파일을 생성하여 학습 누락 방지
    """
    json_dir = Path(json_dir)
    output_dir = Path(output_dir)
    output_dir.mkdir(parents=True, exist_ok=True)
    log_file = Path(log_path)

    json_files = list(json_dir.glob("*.json"))
    log_lines = [f"총 {len(json_files)}개의 JSON 파일을 처리합니다.\n\n"]

    converted_count = 0
    skipped_empty = 0
    skipped_class = 0
    skipped_invalid_bbox = 0
    skipped_no_size = 0

    for json_file in json_files:
        with open(json_file, "r", encoding="utf-8") as f:
            data = json.load(f)

        # a/b 공통으로 파일명/사이즈 얻기
        filename, img_width, img_height = _get_image_meta(data)
        if filename is None or not (isinstance(img_width, int) and isinstance(img_height, int) and img_width > 0 and img_height > 0):
            skipped_no_size += 1
            # 그래도 빈 파일은 생성 (filename이 없으면 json 스템으로 대체)
            out_stem = (Path(filename).stem if filename else json_file.stem)
            (output_dir / f"{out_stem}.txt").write_text("")
            log_lines.append(f"[스킵:사이즈] {json_file.name} (빈 라벨 생성)\n")
            continue

        label_lines = []

        for ann in data.get("annotations", []):
            # 1) 우선 a 포맷의 class 키로 매핑 시도
            class_id_raw = ann.get("class")
            class_id = None
            if class_id_raw in class_map:
                class_id = class_map[class_id_raw]
            else:
                # 2) b 포맷: helmet_off면 class_map['08']로 매핑 (통일)
                if _is_b_helmet_off(ann):
                    if "08" in class_map:
                        class_id = class_map["08"]
                    else:
                        # class_map에 08이 없다면 스킵
                        skipped_class += 1
                        continue
                else:
                    skipped_class += 1
                    continue

            # 3) box/bbox/polygon → bbox로
            xmin = ymin = xmax = ymax = None
            if "box" in ann and isinstance(ann["box"], list) and len(ann["box"]) >= 4:
                xmin, ymin, xmax, ymax = ann["box"][:4]
            elif "bbox" in ann and isinstance(ann["bbox"], list) and len(ann["bbox"]) >= 4:
                xmin, ymin, xmax, ymax = ann["bbox"][:4]
            elif "polygon" in ann and isinstance(ann["polygon"], list) and len(ann["polygon"]) >= 3:
                xs = [pt[0] for pt in ann["polygon"]]
                ys = [pt[1] for pt in ann["polygon"]]
                xmin, xmax = min(xs), max(xs)
                ymin, ymax = min(ys), max(ys)
            else:
                # 지원하지 않는 형식 → 스킵
                continue

            # 유효성 체크/정리
            try:
                xmin = float(xmin); ymin = float(ymin); xmax = float(xmax); ymax = float(ymax)
            except Exception:
                skipped_invalid_bbox += 1
                continue

            if xmax < xmin: xmin, xmax = xmax, xmin
            if ymax < ymin: ymin, ymax = ymax, ymin

            if (xmax - xmin < 1) or (ymax - ymin < 1):
                skipped_invalid_bbox += 1
                continue
            if not (0 <= xmin < xmax <= img_width) or not (0 <= ymin < ymax <= img_height):
                skipped_invalid_bbox += 1
                continue

            # YOLO format 변환
            cx = ((xmin + xmax) / 2) / img_width
            cy = ((ymin + ymax) / 2) / img_height
            w = (xmax - xmin) / img_width
            h = (ymax - ymin) / img_height

            label_lines.append(f"{class_id} {cx:.6f} {cy:.6f} {w:.6f} {h:.6f}")

        # 파일 저장 (비어있어도 생성)
        out_stem = Path(filename).stem
        label_file = output_dir / f"{out_stem}.txt"
        with open(label_file, "w") as out_f:
            if label_lines:
                out_f.write("\n".join(label_lines))
                converted_count += 1
                log_lines.append(f"변환 완료: {filename} → {len(label_lines)}개 객체\n")
            else:
                out_f.write("")
                skipped_empty += 1
                log_lines.append(f"객체 없음 (빈 라벨 생성): {filename}\n")

    # 로그 파일 저장
    with log_file.open("w", encoding="utf-8") as log:
        log.writelines(log_lines)
        log.write("\n요약\n")
        log.write(f"converted_count={converted_count}\n")
        log.write(f"skipped_empty={skipped_empty}\n")
        log.write(f"skipped_class={skipped_class}\n")
        log.write(f"skipped_invalid_bbox={skipped_invalid_bbox}\n")
        log.write(f"skipped_no_size={skipped_no_size}\n")

    # (옵션) 환경 점검 출력
    print("train images exists:", Path("/content/YOLO_safety/train/images").exists())
    print("val images exists:", Path("/content/YOLO_safety/val/images").exists())
    train_labels = list(Path("/content/YOLO_safety/train/labels").glob("*.txt"))
    print(f"train 라벨 수: {len(train_labels)}개")
    val_labels = list(Path("/content/YOLO_safety/val/labels").glob("*.txt"))
    print(f"val 라벨 수: {len(val_labels)}개")
    empty = [f for f in val_labels if f.stat().st_size == 0]
    print(f"비어있는 라벨 파일 수: {len(empty)}개")
    print(f"라벨 변환 로그가 '{log_path}'에 저장되었습니다.")


In [13]:
# 클래스 매핑: JSON 클래스 번호(str) → YOLO 클래스 인덱스(int)
class_mapping_all = {
    "01": 0,  # 안전벨트 착용
    "02": 1,  # 안전벨트 미착용
    "07": 2,  # 안전모 착용
    "08": 3,  # 안전모 미착용 (a)
}

# 학습 변환
convert_box_json_to_yolo(
    json_dir="/content/YOLO_safety/train/labels_json",
    output_dir="/content/YOLO_safety/train/labels",
    class_map=class_mapping_all,
    log_path="train_log.txt"
)
print("학습 데이터 전체 라벨 변환 완료")

# 검증 변환
convert_box_json_to_yolo(
    json_dir="/content/YOLO_safety/val/labels_json",
    output_dir="/content/YOLO_safety/val/labels",
    class_map=class_mapping_all,
    log_path="val_log.txt"
)
print("검증 데이터 전체 라벨 변환 완료")

train images exists: True
val images exists: True
train 라벨 수: 36616개
val 라벨 수: 0개
비어있는 라벨 파일 수: 0개
라벨 변환 로그가 'train_log.txt'에 저장되었습니다.
학습 데이터 전체 라벨 변환 완료
train images exists: True
val images exists: True
train 라벨 수: 36616개
val 라벨 수: 4596개
비어있는 라벨 파일 수: 0개
라벨 변환 로그가 'val_log.txt'에 저장되었습니다.
검증 데이터 전체 라벨 변환 완료


In [ ]:
# from pathlib import Path
# import os
# # 폴더명 소문자로 바꾸기는 기능
# labels_dir = Path("/content/YOLO_safety/train/labels")  # 라벨 폴더 경로 수정

# for txt_file in labels_dir.glob("*.txt"):
#     new_name = txt_file.with_name(txt_file.stem.lower() + ".txt")
#     os.rename(txt_file, new_name)
#     print(f"Renamed: {txt_file.name} → {new_name.name}")

In [ ]:
# from pathlib import Path
# import os

# # 이미지 디렉토리 경로 지정
# train_images_dir = Path("/content/YOLO_safety/train/images")

# def rename_to_lowercase(directory):
#     for img_file in directory.glob("*.*"):
#         lower_name = img_file.with_name(img_file.stem.lower() + img_file.suffix.lower())
#         if img_file != lower_name:  # 이름이 다를 때만 실행
#             os.rename(img_file, lower_name)
#             print(f"Renamed: {img_file.name} → {lower_name.name}")

# # 실행
# print("Train 이미지 파일명 수정:")
# rename_to_lowercase(train_images_dir)



In [14]:
from pathlib import Path

# 디렉토리 설정
train_image_dir = Path("/content/YOLO_safety/train/images")
val_image_dir = Path("/content/YOLO_safety/val/images")

train_label_dir = Path("/content/YOLO_safety/train/labels")
val_label_dir = Path("/content/YOLO_safety/val/labels")

def check_dataset(image_dir, label_dir, dataset_name="Dataset"):
    images = list(image_dir.glob("*.jpg")) + list(image_dir.glob("*.png"))
    labels = list(label_dir.glob("*.txt"))

    image_names = {img.stem for img in images}
    label_names = {lbl.stem for lbl in labels}

    # 매칭 검사
    no_label = image_names - label_names  # 라벨 없는 이미지
    no_image = label_names - image_names  # 이미지 없는 라벨

    print(f"\n{dataset_name} 데이터셋 확인")
    print(f" - 이미지 수: {len(images)}개")
    print(f" - 라벨 수: {len(labels)}개")

    if no_label:
        print(f" 라벨 없는 이미지 {len(no_label)}개 예시: {list(no_label)[:3]}")
    else:
        print(" 모든 이미지에 라벨이 존재")

    if no_image:
        print(f" 이미지 없는 라벨 {len(no_image)}개 예시: {list(no_image)[:3]}")
    else:
        print(" 모든 라벨에 대응되는 이미지 존재")

# 학습 / 검증 데이터셋 확인
check_dataset(train_image_dir, train_label_dir, "학습 (Train)")
check_dataset(val_image_dir, val_label_dir, "검증 (Val)")

# 총합
train_count = len(list(train_label_dir.glob("*.txt")))
val_count = len(list(val_label_dir.glob("*.txt")))
print(f"\n총 라벨 파일 개수: {train_count + val_count}개 (학습 {train_count}개 + 검증 {val_count}개)")



학습 (Train) 데이터셋 확인
 - 이미지 수: 36616개
 - 라벨 수: 36616개
 모든 이미지에 라벨이 존재
 모든 라벨에 대응되는 이미지 존재

검증 (Val) 데이터셋 확인
 - 이미지 수: 4596개
 - 라벨 수: 4596개
 모든 이미지에 라벨이 존재
 모든 라벨에 대응되는 이미지 존재

총 라벨 파일 개수: 41212개 (학습 36616개 + 검증 4596개)


In [15]:
from pathlib import Path
import os

# 이미지 파일명, json 파일명 짝 안맞는거 삭제하는 기능
# 디렉토리 설정
train_image_dir = Path("/content/YOLO_safety/train/images")
val_image_dir = Path("/content/YOLO_safety/val/images")

train_label_dir = Path("/content/YOLO_safety/train/labels")
val_label_dir = Path("/content/YOLO_safety/val/labels")

# JSON 경로 설정 (YOLO 라벨 json 폴더가 있다면)
train_json_dir = Path("/content/YOLO_safety/train/labels_json")
val_json_dir = Path("/content/YOLO_safety/val/labels_json")

def clean_unmatched(image_dir, label_dir, json_dir=None, dataset_name="Dataset"):
    images = list(image_dir.glob("*.jpg")) + list(image_dir.glob("*.png"))
    labels = list(label_dir.glob("*.txt"))

    image_names = {img.stem for img in images}
    label_names = {lbl.stem for lbl in labels}

    # 매칭 검사
    no_label = image_names - label_names  # 라벨 없는 이미지
    no_image = label_names - image_names  # 이미지 없는 라벨

    print(f"\n{dataset_name} 데이터셋 정리 시작")

    # 라벨 없는 이미지 삭제
    for stem in no_label:
        for ext in [".jpg", ".png"]:
            img_path = image_dir / f"{stem}{ext}"
            if img_path.exists():
                os.remove(img_path)
                print(f"이미지 삭제: {img_path}")

        if json_dir:  # JSON도 같이 삭제
            json_path = json_dir / f"{stem}.json"
            if json_path.exists():
                os.remove(json_path)
                print(f"JSON 삭제: {json_path}")

    # 이미지 없는 라벨 삭제
    for stem in no_image:
        txt_path = label_dir / f"{stem}.txt"
        if txt_path.exists():
            os.remove(txt_path)
            print(f"라벨 삭제: {txt_path}")

        if json_dir:
            json_path = json_dir / f"{stem}.json"
            if json_path.exists():
                os.remove(json_path)
                print(f"JSON 삭제: {json_path}")

    print(f"{dataset_name} 정리 완료")

# 실행
clean_unmatched(train_image_dir, train_label_dir, train_json_dir, "학습 (Train)")
clean_unmatched(val_image_dir, val_label_dir, val_json_dir, "검증 (Val)")



학습 (Train) 데이터셋 정리 시작
학습 (Train) 정리 완료

검증 (Val) 데이터셋 정리 시작
검증 (Val) 정리 완료


In [18]:
import os
from collections import Counter

def count_yolo_classes(label_dir, class_names=None):
    """
    YOLO 라벨(.txt) 파일에서 클래스별 인스턴스 수를 카운트합니다.

    :param label_dir: 라벨 파일(.txt)들이 있는 디렉토리
    :param class_names: 클래스 이름 리스트 (있으면 이름과 함께 출력됨)
    """
    counter = Counter()

    for filename in os.listdir(label_dir):
        if filename.endswith(".txt"):
            with open(os.path.join(label_dir, filename), "r", encoding="utf-8") as f:
                for line in f:
                    class_id = line.strip().split()[0]  # 첫 번째 값이 class_id
                    counter[int(class_id)] += 1

    print("클래스별 인스턴스 수:")
    for class_id, count in sorted(counter.items()):
        name = class_names[class_id] if class_names else f"class_{class_id}"
        print(f"  {name} ({class_id}): {count}개")

    return counter

label_dir = "/content/YOLO_safety/train/labels"  # 또는 val
class_names = ["harness_on", "harness_off", 'helmet_on', "helmet_off"]
count_yolo_classes(label_dir, class_names)

클래스별 인스턴스 수:
  harness_on (0): 24436개
  harness_off (1): 26033개
  helmet_on (2): 40907개
  helmet_off (3): 19519개


Counter({2: 40907, 0: 24436, 1: 26033, 3: 19519})

# 이미지 경로 평탄화

In [19]:
import shutil
from pathlib import Path

def flatten_image_dir_skip_duplicates(img_base_dir: Path):
    flat_img_dir = img_base_dir

    for subdir in img_base_dir.iterdir():
        if subdir.is_dir():
            for img_file in subdir.glob("*.jpg"):
                target_path = flat_img_dir / img_file.name

                if target_path.exists():
                    print(f"⚠️ 중복으로 건너뜀: {img_file.name}")
                    continue

                shutil.move(str(img_file), str(target_path))

    print(f"중복 건너뛰며 평탄화 완료: {img_base_dir}")

# 적용
flatten_image_dir_skip_duplicates(Path("/content/YOLO_safety/train/images"))
flatten_image_dir_skip_duplicates(Path("/content/YOLO_safety/val/images"))

중복 건너뛰며 평탄화 완료: /content/YOLO_safety/train/images
중복 건너뛰며 평탄화 완료: /content/YOLO_safety/val/images


# 데이터 존재 확인

In [20]:
from pathlib import Path

# 이미지 디렉토리 존재 확인
print("train images exists:", Path("/content/YOLO_safety/train/images").exists())
print("val images exists:", Path("/content/YOLO_safety/val/images").exists())

# 라벨 디렉토리 + 파일 개수 확인
val_labels = list(Path("/content/YOLO_safety/val/labels").glob("*.txt"))
print(f"val 라벨 수: {len(val_labels)}개")

# 비어있는 파일 확인
empty = [f for f in val_labels if f.stat().st_size == 0]
print(f"비어있는 라벨 파일 수: {len(empty)}개")

train images exists: True
val images exists: True
val 라벨 수: 4596개
비어있는 라벨 파일 수: 0개


In [21]:
# 잘못된 클래스 ID가 있는지 확인
bad_lines = 0
for f in val_labels:
    lines = f.read_text().splitlines()
    for line in lines:
        parts = line.strip().split()
        if not parts:
            continue
        try:
            class_id = int(parts[0])
            if class_id not in [0,1,2,3]:
                print(f"잘못된 클래스 ID: {class_id} in {f.name}")
                bad_lines += 1
        except:
            print(f"형식 오류 in {f.name}: {line}")
            bad_lines += 1

print(f"잘못된 라벨 줄 수: {bad_lines}")

잘못된 라벨 줄 수: 0


In [22]:
# 라벨 파일 몇 개 출력해보기 (예: 처음 3개)
for f in val_labels[:3]:
    print(f"\n📄 파일명: {f.name}")
    print(f.read_text())


📄 파일명: S2-N0108M23636.txt
2 0.500000 0.378241 0.159375 0.323148
2 0.593750 0.565278 0.055208 0.097222
1 0.486198 0.766204 0.247396 0.467593

📄 파일명: S2-N0109M06641.txt
2 0.246615 0.419444 0.033854 0.075926
2 0.410156 0.385185 0.030729 0.075926
2 0.627604 0.400463 0.037500 0.087963
2 0.708073 0.392593 0.033854 0.077778
2 0.974740 0.391667 0.026562 0.057407
0 0.250000 0.500000 0.047917 0.127778
0 0.415365 0.475000 0.046354 0.131481
0 0.609635 0.481481 0.046354 0.125926
0 0.712240 0.494444 0.045312 0.142593
1 0.976823 0.450000 0.036979 0.096296

📄 파일명: S2-N2103M49055.txt
2 0.364844 0.246296 0.058854 0.124074
2 0.541406 0.205556 0.056771 0.116667
2 0.682813 0.237037 0.052083 0.112963
0 0.350000 0.427315 0.094792 0.269444
0 0.543490 0.401389 0.097396 0.271296
1 0.673438 0.413426 0.105208 0.226852


In [23]:
from pathlib import Path

val_labels = list(Path("/content/YOLO_safety/val/labels").glob("*.txt"))

# 0번 클래스 포함된 파일 찾기
person_labels = []
for f in val_labels:
    lines = f.read_text().splitlines()
    if any(line.strip().startswith("1") for line in lines):  # class_id == 0
        person_labels.append(f)

# 최대 3개만 출력
for f in person_labels[:3]:
    print(f"\n파일명: {f.name}")
    print(f.read_text())

if not person_labels:
    print("val 라벨에서 0번 클래스(person)를 포함한 파일을 찾을 수 없습니다.")


파일명: S2-N0108M23636.txt
2 0.500000 0.378241 0.159375 0.323148
2 0.593750 0.565278 0.055208 0.097222
1 0.486198 0.766204 0.247396 0.467593

파일명: S2-N0109M06641.txt
2 0.246615 0.419444 0.033854 0.075926
2 0.410156 0.385185 0.030729 0.075926
2 0.627604 0.400463 0.037500 0.087963
2 0.708073 0.392593 0.033854 0.077778
2 0.974740 0.391667 0.026562 0.057407
0 0.250000 0.500000 0.047917 0.127778
0 0.415365 0.475000 0.046354 0.131481
0 0.609635 0.481481 0.046354 0.125926
0 0.712240 0.494444 0.045312 0.142593
1 0.976823 0.450000 0.036979 0.096296

파일명: S2-N2103M49055.txt
2 0.364844 0.246296 0.058854 0.124074
2 0.541406 0.205556 0.056771 0.116667
2 0.682813 0.237037 0.052083 0.112963
0 0.350000 0.427315 0.094792 0.269444
0 0.543490 0.401389 0.097396 0.271296
1 0.673438 0.413426 0.105208 0.226852


In [24]:
val_img_dir = Path("/content/YOLO_safety/val/images")
val_images = [f.stem for f in val_img_dir.glob("*.jpg")]
val_labels = [f.stem for f in Path("/content/YOLO_safety/val/labels").glob("*.txt")]

no_label_images = set(val_images) - set(val_labels)

print(f"라벨이 없는 이미지 수: {len(no_label_images)}")
print(list(no_label_images)[:5])  # 일부 예시

라벨이 없는 이미지 수: 0
[]


In [25]:
import os

# 존재 확인
print("val 이미지 디렉토리 있음:", os.path.exists("/content/YOLO_safety/val/images"))
print("val 라벨 디렉토리 있음:", os.path.exists("/content/YOLO_safety/val/labels"))

val 이미지 디렉토리 있음: True
val 라벨 디렉토리 있음: True


# 데이터 증강

In [26]:
import sys
sys.path.append('/content/drive/MyDrive')
from augmentations import apply_custom_augmentations

In [27]:
!ls /content

drive  sample_data  train_log.txt  val_log.txt	YOLO_safety


In [28]:
# /content/augmentations.py
import cv2
import numpy as np
import random

def apply_custom_augmentations(im, hyp):
    # Grayscale
    if hyp.get('grayscale', 0) > 0 and random.random() < hyp['grayscale']:
        im = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)
        im = cv2.cvtColor(im, cv2.COLOR_GRAY2BGR)

    # Brightness
    if hyp.get('brightness', 0) > 0 and random.random() < hyp['brightness']:
        factor = 1.0 + np.random.uniform(-0.2, 0.2)
        im = np.clip(im * factor, 0, 255).astype(np.uint8)

    # Gamma correction
    if hyp.get('gamma', 0) > 0 and random.random() < hyp['gamma']:
        gamma = np.random.uniform(0.7, 1.5)
        inv_gamma = 1.0 / gamma
        table = np.array([((i / 255.0) ** inv_gamma) * 255
                          for i in np.arange(256)]).astype("uint8")
        im = cv2.LUT(im, table)

    # CLAHE
    if hyp.get('clahe', 0) > 0 and random.random() < hyp['clahe']:
        lab = cv2.cvtColor(im, cv2.COLOR_BGR2LAB)
        l, a, b = cv2.split(lab)
        clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
        cl = clahe.apply(l)
        merged = cv2.merge((cl, a, b))
        im = cv2.cvtColor(merged, cv2.COLOR_LAB2BGR)

    # Blur
    if hyp.get('blur', 0) > 0 and random.random() < hyp['blur']:
        k = random.choice([3, 5])  # 3x3 or 5x5 kernel
        im = cv2.GaussianBlur(im, (k, k), 0)

    # Gaussian Noise
    if hyp.get('noise', 0) > 0 and random.random() < hyp['noise']:
        noise = np.random.normal(0, 25, im.shape).astype(np.int16)
        im = np.clip(im.astype(np.int16) + noise, 0, 255).astype(np.uint8)

    return im


In [29]:
hyp_text = """
# /content/YOLO_safety/hyp_cctv.yaml

# 증강 - 좌우/상하 반전
fliplr: 0.3
flipud: 0.0

# 증강 - Mosaic & Mixup
mosaic: 0.6

# 증강 - 색상 계열 (HSV)
hsv_h: 0.015
hsv_s: 0.7
hsv_v: 0.4

# 증강 - 기하학 왜곡
translate: 0.2
scale: 0.2
"""
with open("/content/YOLO_safety/hyp_cctv.yaml", "w") as f:
    f.write(hyp_text)

print("hyp_cctv.yaml 저장 완료")


hyp_cctv.yaml 저장 완료


# 학습

In [30]:
yaml_text = """
path: /content/YOLO_safety
train: train/images
val: val/images

names: [harness_on, harness_off, helmet_on, helmet_off]
"""

with open("/content/YOLO_safety/safety.yaml", "w") as f:
    f.write(yaml_text)

print("safety.yaml 저장 완료")

safety.yaml 저장 완료


## 증강 적용 X

In [ ]:
# from ultralytics import YOLO
# model = YOLO("yolov8n.pt")  # 또는 yolov8s.pt -> yolov8n.pt -> yolov8m.pt

# model.train(
#     data="/content/YOLO_safety/safety.yaml",            # 데이터셋 구성정보가 담긴 yaml
#     epochs=50,                                          # 전체 학습 반복 횟수
#     imgsz=640,                                          # 학습에 사용할 입력 이미지 크기
#     project="/content/drive/MyDrive/YOLO_safety/runs",  # 실험 결과 저장 경로
#     name="safetyGear_detect_exp1"                       # 실험명
# )


## 증강 적용 O

In [31]:
import sys
sys.path.append('/content/drive/MyDrive')
from augmentations import apply_custom_augmentations

In [32]:
import yaml
# hyp.yaml을 딕셔너리로 로드
with open("/content/YOLO_safety/hyp_cctv.yaml", "r") as f:
    hyp_dict = yaml.safe_load(f)

추가

In [ ]:
# %%writefile /content/augment_match_to_class3.py
# import os, cv2, random
# from pathlib import Path
# from collections import Counter
# from augmentations import apply_custom_augmentations  # 여기!

# #증강 적용
# ROOT = Path("/content/YOLO_safety")
# SPLITS = ["train", "val"]
# IMG_DIR = "images"
# LBL_DIR = "labels"

# NUM_CLASSES = 4
# TARGET_CLASS = 2   # 0:harness_on, 1:harness_off, 2:helmet_on, 3:helmet_off
# ONE_BASED = False  # 라벨이 1..4로 저장돼 있으면 True

# SEED = 42
# MAX_PER_IMAGE = 1
# SAVE_SUFFIX_FMT = "_aug{:03d}"
# random.seed(SEED)

# HYP = dict(grayscale=0.3, brightness=0.8, gamma=0.5, clahe=0.4, blur=0.3, noise=0.4)
# IMG_EXTS = [".jpg",".jpeg",".png",".bmp",".webp",".JPG",".JPEG",".PNG",".BMP",".WEBP"]

# def read_yolo(lbl_path: Path):
#     items = []
#     if not lbl_path.exists():
#         return items
#     for ln in lbl_path.read_text().splitlines():
#         if not ln.strip():
#             continue
#         parts = ln.split()
#         try:
#             cid = int(parts[0])
#             if ONE_BASED:
#                 cid -= 1
#         except:
#             continue
#         if 0 <= cid < NUM_CLASSES:
#             items.append((cid, *map(float, parts[1:5])))
#     return items

# def count_vec(lbl_path: Path):
#     v = [0]*NUM_CLASSES
#     for (cid, *_bbox) in read_yolo(lbl_path):
#         v[cid] += 1
#     return v

# def find_image(img_dir: Path, stem: str):
#     for ext in IMG_EXTS:
#         p = img_dir / f"{stem}{ext}"
#         if p.exists():
#             return p
#     return None

# def write_aug(img_p: Path, lbl_p: Path, aug_idx: int):
#     img = cv2.imread(str(img_p))
#     if img is None:
#         return False
#     aug = apply_custom_augmentations(img, HYP)
#     out_stem = img_p.stem + SAVE_SUFFIX_FMT.format(aug_idx)
#     out_img = img_p.parent / f"{out_stem}{img_p.suffix}"
#     out_lbl = lbl_p.parent / f"{out_stem}.txt"
#     cv2.imwrite(str(out_img), aug)
#     out_lbl.write_text(lbl_p.read_text())
#     return True

# def plan_and_augment(split: str):
#     img_dir = ROOT / split / IMG_DIR
#     lbl_dir = ROOT / split / LBL_DIR
#     assert img_dir.exists() and lbl_dir.exists(), f"경로 확인: {img_dir} / {lbl_dir}"

#     per_img = {}
#     total = [0]*NUM_CLASSES
#     for p in lbl_dir.glob("*.txt"):
#         v = count_vec(p)
#         if sum(v) == 0: continue
#         per_img[p] = v
#         for i in range(NUM_CLASSES):
#             total[i] += v[i]

#     print(f"=== {split} ===")
#     for i,c in enumerate(total): print(f"class {i}: {c}")
#     target = total[TARGET_CLASS]
#     needs = [max(0, target - total[i]) if i != TARGET_CLASS else 0 for i in range(NUM_CLASSES)]
#     if sum(needs)==0:
#         print("균형 OK. 증강 생략"); return
#     print("needs:", {i:n for i,n in enumerate(needs) if n>0})

#     pure_q = {k: [] for k in range(NUM_CLASSES)}
#     mix_q  = {k: [] for k in range(NUM_CLASSES)}
#     for lbl_p, vec in per_img.items():
#         img_p = find_image(img_dir, lbl_p.stem)
#         if not img_p: continue
#         total_inst = sum(vec)
#         for k in range(NUM_CLASSES):
#             if k == TARGET_CLASS: continue
#             if vec[k] > 0:
#                 others = total_inst - vec[k]
#                 if others == 0:
#                     pure_q[k].append((lbl_p, img_p, vec))
#                 else:
#                     mix_q[k].append((lbl_p, img_p, vec, others))

#     for k in range(NUM_CLASSES):
#         mix_q[k].sort(key=lambda x: (x[3], -x[2][k]))

#     aug_id_per_img = Counter()
#     added = [0]*NUM_CLASSES
#     for k in range(NUM_CLASSES):
#         need = needs[k]
#         if need <= 0: continue
#         made = 0

#         # (a) 순수 이미지 우선
#         for (lbl_p, img_p, vec) in pure_q[k]:
#             if made >= need: break
#             per = vec[k]
#             times = min(MAX_PER_IMAGE, max(1, (need - made + per - 1) // per))
#             for _ in range(times):
#                 if made >= need: break
#                 aidx = aug_id_per_img[img_p]; aug_id_per_img[img_p] += 1
#                 if write_aug(img_p, lbl_p, aidx):
#                     made += per
#                     added[k] += per

#         # (b) 혼합 이미지 보충
#         if made < need:
#             for (lbl_p, img_p, vec, _others) in mix_q[k]:
#                 if made >= need: break
#                 aidx = aug_id_per_img[img_p]; aug_id_per_img[img_p] += 1
#                 if write_aug(img_p, lbl_p, aidx):
#                     made += vec[k]
#                     for j in range(NUM_CLASSES):
#                         added[j] += vec[j]

#         print(f"[{split}] class {k}: need {need} → added ≈{made}")

#     print(f"[{split}] 증가 추정:", {i:added[i] for i in range(NUM_CLASSES)})
#     print(f"저장: {img_dir} / {lbl_dir} (*_aug###.jpg/.txt)")

# if __name__ == "__main__":
#     for sp in ["train","val"]:
#         plan_and_augment(sp)


In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

# # 실제 위치에 맞게 경로 수정
# !cp -f "/content/drive/MyDrive/augmentations.py" /content/augmentations.py
# # 또는 예: !cp -f "/content/drive/MyDrive/YOLO_safety/augmentations.py" /content/augmentations.py

# # 실행
# !python /content/augment_match_to_class3.py

In [ ]:
# from pathlib import Path
# from collections import Counter
# # 코드 인스턴스 수 재검증
# def count_yolo_instances(label_dir, one_based=False, nc=4):
#     cnt = Counter()
#     for p in Path(label_dir).glob("*.txt"):
#         for ln in p.read_text().splitlines():
#             if not ln.strip():
#                 continue
#             try:
#                 cid = int(ln.split()[0])
#                 if one_based: cid -= 1
#                 if 0 <= cid < nc:
#                     cnt[cid] += 1
#             except:
#                 pass
#     # 누락 방지용: 없는 클래스도 0으로 채움
#     return {i: cnt.get(i, 0) for i in range(nc)}

# for split in ["train", "val"]:
#     d = f"/content/YOLO_safety/{split}/labels"
#     cnt = count_yolo_instances(d, one_based=False, nc=4)  # 0-based 라벨 가정
#     target = cnt[2]  # helmet_on
#     print(f"\n=== {split} ===")
#     for i,name in enumerate(["harness_on","harness_off","helmet_on","helmet_off"]):
#         delta = target - cnt[i] if i!=2 else 0
#         print(f"class {i} ({name}): {cnt[i]}  " + (f"→ 부족 {delta}" if i!=2 and delta>0 else ""))


In [ ]:
# # ===== Train only (default aug ON + 네 hyp 적용) =====
# from ultralytics import YOLO
# import yaml, torch

# DATA_YAML = "/content/YOLO_safety/safety.yaml"           # names: [harness_on, harness_off, helmet_on, helmet_off] (0-based)
# HYP_YAML  = "/content/YOLO_safety/hyp_cctv.yaml"         # 네가 만든 증강/하이퍼
# PROJECT   = "/content/drive/MyDrive/YOLO_safety/runs"
# RUN_NAME  = "yolov8m_baseline"

# # (선택) 장치 확인
# print("Device:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "CPU")

# # hyp 로드(없으면 기본값으로 진행)
# try:
#     with open(HYP_YAML, "r") as f:
#         yolo_hyp = yaml.safe_load(f) or {}
# except FileNotFoundError:
#     yolo_hyp = {}
#     print("hyp_cctv.yaml이 없어 기본 하이퍼로 진행합니다.")

# # 모델 로드 (필요시 n/s/l/x로 교체)
# model = YOLO("yolov8m.pt")

# # 학습
# results = model.train(
#     data=DATA_YAML,
#     epochs=50,
#     imgsz=960,
#     batch=-1,               # VRAM에 맞춰 자동 배치
#     device=0,               # GPU
#     project=PROJECT,
#     name=RUN_NAME,
#     patience=12,            # 조기 종료 여유
#     save_period=5,          # 체크포인트 주기
#     cache=True,             # 데이터 캐시로 I/O 속도↑
#     seed=42,                # 재현성
#     optimizer="AdamW",      # 일반적으로 수렴 안정적
#     cos_lr=True,            # cosine LR 스케줄(완만한 수렴)
#     close_mosaic=10,        # 마지막 10epoch 모자이크 OFF로 안정화
#     amp=True,               # 혼합정밀로 속도/메모리↑
#     **yolo_hyp              # 네가 만든 증강/하이퍼 적용(기본 증강 ON)
# )

# # (선택) 최종 검증 한번 더
# model.val(data=DATA_YAML, imgsz=960, device=0, split="val")


In [33]:
from ultralytics import YOLO
import yaml, cv2, torch
from augmentations import apply_custom_augmentations

# 증강 하이퍼파라미터 (외부 증강용)
custom_hyp = {
    "grayscale": 0.05,
    "brightness": 0.1,
    "gamma": 0.05,
    "clahe": 0.05,
    "blur": 0.1,
    "noise": 0.1,
}

# YOLO 내부 증강 하이퍼파라미터 로드
with open("/content/YOLO_safety/hyp_cctv.yaml", "r") as f:
    yolo_hyp = yaml.safe_load(f)

# 모델 로드
model = YOLO("yolov8m.pt")

#  외부 증강 콜백 정의
def custom_callback(trainer):
    # train 단계에서만 적용
    if trainer.mode != "train":
        return

    images = trainer.batch[0]
    for i in range(len(images)):
        im = images[i].permute(1,2,0).cpu().numpy()
        im = cv2.cvtColor(im, cv2.COLOR_RGB2BGR)

        # 외부 증강 적용
        im = apply_custom_augmentations(im, custom_hyp)

        im = cv2.cvtColor(im, cv2.COLOR_BGR2RGB)
        images[i] = torch.from_numpy(im).permute(2,0,1)

# 콜백 등록
model.add_callback("on_batch_start", custom_callback)

# 모델 학습
model.train(
    data="/content/YOLO_safety/safety.yaml",
    batch=-1,               # VRAM에 맞춰 자동 배치
    epochs=50,
    imgsz=960,
    cache=True,             # 데이터 캐시로 I/O 속도↑
    cos_lr=True,            # cosine LR 스케줄(완만한 수렴)
    close_mosaic=10,        # 마지막 10epoch 모자이크 OFF로 안정화
    amp=True,               # 혼합정밀로 속도/메모리↑
    project="/content/drive/MyDrive/YOLO_safety/runs",
    name="yolov8m_plus",
    patience=15,
    resume=False,
    save_period=5,
    **yolo_hyp  # YOLO 내부 증강
)


Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


Ultralytics 8.3.179 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (NVIDIA A100-SXM4-40GB, 40507MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=-1, bgr=0.0, box=7.5, cache=True, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=True, cutmix=0.0, data=/content/YOLO_safety/safety.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=False, fliplr=0.3, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=960, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8m.pt, momentum=0.937, mosaic=0.6, multi_scale=False, name=yolov8m_plus4, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=15, perspective=0.0, plots=True,

Overriding model.yaml nc=80 with nc=4

                   from  n    params  module                                       arguments                     
  0                  -1  1      1392  ultralytics.nn.modules.conv.Conv             [3, 48, 3, 2]                 
  1                  -1  1     41664  ultralytics.nn.modules.conv.Conv             [48, 96, 3, 2]                
  2                  -1  2    111360  ultralytics.nn.modules.block.C2f             [96, 96, 2, True]             
  3                  -1  1    166272  ultralytics.nn.modules.conv.Conv             [96, 192, 3, 2]               
  4                  -1  4    813312  ultralytics.nn.modules.block.C2f             [192, 192, 4, True]           
  5                  -1  1    664320  ultralytics.nn.modules.conv.Conv             [192, 384, 3, 2]              
  6                  -1  4   3248640  ultralytics.nn.modules.block.C2f             [384, 384, 4, True]           
  7                  -1  1   1991808  ultralytics

 12                  -1  2   1993728  ultralytics.nn.modules.block.C2f             [960, 384, 2]                 
 13                  -1  1         0  torch.nn.modules.upsampling.Upsample         [None, 2, 'nearest']          
 14             [-1, 4]  1         0  ultralytics.nn.modules.conv.Concat           [1]                           
 15                  -1  2    517632  ultralytics.nn.modules.block.C2f             [576, 192, 2]                 
 16                  -1  1    332160  ultralytics.nn.modules.conv.Conv             [192, 192, 3, 2]              
 17            [-1, 12]  1         0  ultralytics.nn.modules.conv.Concat           [1]                           
 18                  -1  2   1846272  ultralytics.nn.modules.block.C2f             [576, 384, 2]                 
 19                  -1  1   1327872  ultralytics.nn.modules.conv.Conv             [384, 384, 3, 2]              
 20             [-1, 9]  1         0  ultralytics.nn.modules.conv.Concat           [1]  

AMP: checks passed ✅
train: Fast image access ✅ (ping: 0.0±0.0 ms, read: 1953.2±1523.0 MB/s, size: 665.4 KB)


...

                   all       4596      12551      0.897      0.862       0.91      0.571
EarlyStopping: Training stopped early as no improvement observed in last 15 epochs. Best results observed at epoch 20, best model saved as best.pt.
To update EarlyStopping(patience=15) pass a new patience value, i.e. `patience=300` or use `patience=0` to disable EarlyStopping.



35 epochs completed in 5.124 hours.
Optimizer stripped from /content/drive/MyDrive/YOLO_safety/runs/yolov8m_plus4/weights/last.pt, 52.0MB
Optimizer stripped from /content/drive/MyDrive/YOLO_safety/runs/yolov8m_plus4/weights/best.pt, 52.0MB

Validating /content/drive/MyDrive/YOLO_safety/runs/yolov8m_plus4/weights/best.pt...
Ultralytics 8.3.179 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (NVIDIA A100-SXM4-40GB, 40507MiB)
Model summary (fused): 92 layers, 25,842,076 parameters, 0 gradients, 78.7 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 105/105 [00:26<00:00,  4.01it/s]


                   all       4596      12551      0.895      0.864      0.922       0.58
            harness_on       1208       2462      0.925      0.882      0.933      0.567
           harness_off       2574       3318      0.867      0.827      0.898      0.498
             helmet_on       2236       4470      0.897      0.933      0.949      0.703
            helmet_off       2252       2301       0.89      0.812      0.907      0.553
Speed: 0.2ms preprocess, 1.9ms inference, 0.0ms loss, 1.0ms postprocess per image
Results saved to /content/drive/MyDrive/YOLO_safety/runs/yolov8m_plus4


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1, 2, 3])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7dcf590c4310>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0

In [ ]:
# from ultralytics import YOLO

# model = YOLO("/content/drive/MyDrive/YOLO_safety/runs/Yolov8m_model_1747/weights/last.pt")

# # 학습 실행
# model.train(
#     data="/content/YOLO_safety/safety.yaml",               # 데이터셋 구성 정보
#     epochs=50,                                             # 전체 epoch 수
#     imgsz=960,                                              # 입력 이미지 크기
#     project="/content/drive/MyDrive/YOLO_safety/runs",     # 결과 저장 경로
#     name="Yolov8m_model_many",                              # 실험 이름

#     # 성능 향상을 위한 사용자 지정 값
#     optimizer='AdamW',       # 더 나은 수렴 속도와 일반화
#     cos_lr=True,             # 부드러운 cosine 학습률 스케줄링
#     patience=20,             # EarlyStopping 인내심
#     seed=42,                 # 재현성 확보
#     save_period=5,          # 10 epoch마다 중간 모델 저장

#     **hyp_dict               # 사용자 정의 하이퍼파라미터 (flip, hsv 등)
# )